In [1]:
import plotly.express as px
from scipy.stats import binom
import pandas as pd
import csv
import numpy as np
import datetime
import re
import warnings
import sys
from scipy.optimize import curve_fit
from math import pi
from scipy import stats
from scipy.stats import lognorm
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)

In [2]:
exclude = ['Recovered', 'Grand Princess', 'Diamond Princess', 'American Samoa', 'US', 
           'American Samoa', 'Northern Mariana Islands', 'Puerto Rico', 'Guam', 
           'Virgin Islands']

df0 = pd.read_csv('data/COVID-TESTING-DF.txt', sep='\t')
df0 = df0.loc[~df0['Province_State'].isin(exclude)]
states = df0['Province_State'].tolist()
JH_dates = df0['date'].tolist()

states_df = pd.read_csv('data/StatePops.csv', sep=',')
states_df = states_df.loc[~states_df['Province/State'].isin(exclude)]

fits_df = pd.read_pickle('data/model_results_dataframe.pkl')
fits_df = fits_df[fits_df['model'] == 'Quadratic'] 
fits_df = fits_df[fits_df['label'] == 'Current forecast']

AA_df = pd.read_csv('data/African_American.csv', sep=',')
AA_df = AA_df.loc[~AA_df['State'].isin(exclude)]

Poverty_df = pd.read_csv('data/Poverty.csv', sep=',')
Poverty_df = Poverty_df.loc[~Poverty_df['State'].isin(exclude)]

In [3]:
exclude_abbv = ['AS', 'VI', 'MP', 'GU', 'PR']
Atlantic_df = pd.read_csv('https://raw.githubusercontent.com/COVID19Tracking/covid-tracking-data/master/data/states_daily_4pm_et.csv', sep=',')
Atlantic_df.drop(['hash', 'dateChecked', 'fips', 'posNeg'], axis=1, inplace=True)
Atlantic_df = Atlantic_df.loc[~Atlantic_df['state'].isin(exclude_abbv)]
state_abvs = Atlantic_df['state'].tolist()

In [4]:
dates = Atlantic_df['date'].tolist()
dates_reformat = []
for d1 in dates:
    d2 = datetime.datetime.strptime(str(d1), '%Y%m%d')
    d2 = d2.strftime("%m/%d/%y")
    dates_reformat.append(d2)
    
Atlantic_df['formatted_dates'] = dates_reformat
Atlantic_df['UniqueRow'] = Atlantic_df['formatted_dates'] + '-' + Atlantic_df['state']

In [5]:
pop_sizes = []
colors = []
quadratic_r2s = []
AA_pop_tot = []
AA_pop_per = []
per_poor = []
unique_rows = []
delta_testing_rate = []
for i, state in enumerate(states):
    
    df_sub = df0[df0['Province_State'] == state]
    testing_rate = df_sub['Testing_Rate'].tolist()
    
    x = list(range(len(testing_rate)))
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, testing_rate)
    delta_testing_rate.append(slope)

    state_abv = states_df[states_df['Province/State'] == state]['Abbreviation'].iloc[0]
    u_row = JH_dates[i] + '-' + state_abv
    unique_rows.append(u_row)
    
    pop_size = states_df[states_df['Province/State'] == state].PopSize.iloc[0]
    pop_sizes.append(pop_size)
    
    r2 = fits_df[fits_df['focal_loc'] == state]['obs_pred_r2'].iloc[0]
    quadratic_r2s.append(r2)
    
    black_pop = AA_df[AA_df['State'] == state]['BlackTotal'].iloc[0]
    AA_pop_tot.append(black_pop)
    
    black_pop_per = AA_df[AA_df['State'] == state]['BlackPerc'].iloc[0]
    AA_pop_per.append(black_pop_per)
    
    poor = Poverty_df[Poverty_df['State'] == state]['2017_2018_avg'].iloc[0]
    per_poor.append(poor)
    
    if state == 'Illinois':
        colors.append('#FECB52')
    else:
        colors.append('#636EFA')
        
    
df0['PopSize'] = pop_sizes
df0['color'] = colors
df0['quadratic'] = quadratic_r2s
df0['BlackTotal'] = AA_pop_tot
df0['%Black'] = AA_pop_per
df0['%Poor'] = per_poor
df0['UniqueRow'] = unique_rows
df0['DeltaTestingRate'] = delta_testing_rate

In [6]:
main_df = pd.merge(df0, Atlantic_df, on='UniqueRow')
main_df.tail(10)
main_df.columns = main_df.columns.str.replace('date_x','date')

In [11]:
main_df['sqrt_PopSize'] = np.sqrt(main_df['PopSize'].tolist()).tolist()
main_df['Positives per capita'] = main_df.positive/main_df.PopSize
main_df['Negatives per capita'] = main_df.negative/main_df.PopSize
main_df['Percent positive'] = np.round(100 * main_df.positive/main_df.totalTestResults, 2)
main_df['Tests per capita'] = main_df.People_Tested/main_df.PopSize

main_df['log_PopSize'] = np.log10(main_df['PopSize'])
main_df['log_People_Tested'] = np.log10(main_df['People_Tested'])
main_df['log_Confirmed'] = np.log10(main_df['Confirmed'])
main_df['log_negative'] = np.log10(main_df['negative'])
main_df['log_positive'] = np.log10(main_df['positive'])
main_df['log_hospitalizedCurrently'] = np.log10(main_df['hospitalizedCurrently'])
main_df['log_inIcuCurrently'] = np.log10(main_df['inIcuCurrently'])

try:
    main_df.drop(['Unnamed: 0'], axis=1, inplace=True)
except:
    pass
main_df.head(10)

,Province_State,date,Confirmed,Deaths,Recovered,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate,PopSize,color,quadratic,BlackTotal,%Black,%Poor,UniqueRow,DeltaTestingRate,date_y,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,death,hospitalized,total,totalTestResults,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,formatted_dates,sqrt_PopSize,Positives per capita,Negatives per capita,Percent positive,Tests per capita,log_PopSize,log_People_Tested,log_Confirmed,log_negative,log_positive,log_hospitalizedCurrently,log_inIcuCurrently,testing_rate_incident_rate_ratio
0,Alabama,04/12/20,3563,93,NaN,75.988020,21583.0,437.0,2.610160,460.300152,12.264945,4903185,#636EFA,0.991651,1285737,0.2643,15.6,04/12/20-AL,77.454959,20200412,AL,3525.0,18058.0,NaN,NaN,437.0,NaN,189.0,NaN,119.0,NaN,NaN,4/12/2020 00:00,93.0,437.0,21583,21583,2.0,35.0,0.0,334.0,334.0,04/12/20,2214.313663,0.000719,0.003683,16.33,0.004402,6.690478,4.334112,3.551816,4.256670,3.547159,NaN,NaN,6.057536
1,Alaska,04/12/20,272,8,66.0,45.504049,8038.0,31.0,2.941176,1344.711576,11.397059,731545,#636EFA,0.973847,22817,0.0309,12.6,04/12/20-AK,121.101855,20200412,AK,272.0,7766.0,NaN,NaN,31.0,NaN,NaN,NaN,NaN,66.0,NaN,4/12/2020 14:50,8.0,31.0,8038,8038,0.0,0.0,291.0,15.0,306.0,04/12/20,855.304040,0.000372,0.010616,3.38,0.010988,5.864241,3.905148,2.434569,3.890197,2.434569,NaN,NaN,29.551471
2,Arizona,04/12/20,3542,115,NaN,48.662422,42109.0,NaN,3.246753,578.522286,NaN,7278717,#636EFA,0.995076,286614,0.0413,13.2,04/12/20-AZ,29.730895,20200412,AZ,3539.0,38570.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/12/2020 00:00,115.0,NaN,42109,42109,7.0,0.0,1433.0,146.0,1579.0,04/12/20,2697.909746,0.000486,0.005299,8.40,0.005785,6.862055,4.624375,3.549249,4.586250,3.548881,NaN,NaN,11.888481
3,Arkansas,04/12/20,1280,27,367.0,49.439423,19722.0,130.0,2.109375,761.753354,10.156250,3017804,#636EFA,0.992340,458536,0.1533,15.4,04/12/20-AR,67.132028,20200412,AR,1280.0,18442.0,NaN,74.0,130.0,NaN,43.0,30.0,39.0,367.0,NaN,4/12/2020 11:33,27.0,130.0,19722,19722,3.0,0.0,1090.0,54.0,1144.0,04/12/20,1737.182777,0.000424,0.006111,6.49,0.006535,6.479691,4.294951,3.107210,4.265808,3.107210,1.869232,NaN,15.407813
4,California,04/12/20,22795,640,NaN,58.137726,190328.0,5234.0,2.812020,485.423869,22.961176,39512223,#636EFA,0.996806,2164519,0.0553,12.0,04/12/20-CA,71.091397,20200412,CA,21794.0,168534.0,13200.0,5234.0,NaN,1539.0,NaN,NaN,NaN,NaN,NaN,4/11/2020 00:00,651.0,NaN,203528,190328,42.0,0.0,15930.0,2322.0,18252.0,04/12/20,6285.874879,0.000552,0.004265,11.45,0.004817,7.596731,5.279503,4.357840,5.226688,4.338337,3.718834,3.187239,8.349550
5,Colorado,04/12/20,7307,289,NaN,128.943729,34873.0,1376.0,3.955112,615.389991,18.831258,5758736,#636EFA,0.996677,216732,0.0392,9.0,04/12/20-CO,40.005573,20200412,CO,6893.0,27980.0,NaN,822.0,1376.0,NaN,NaN,NaN,NaN,NaN,NaN,4/11/2020 00:00,274.0,1376.0,34873,34873,24.0,64.0,1837.0,383.0,2220.0,04/12/20,2399.736652,0.001197,0.004859,19.77,0.006056,6.760327,4.542489,3.863739,4.446848,3.838408,2.914872,NaN,4.772547
6,Connecticut,04/12/20,12035,554,NaN,337.560483,41220.0,1654.0,4.603241,1156.148159,13.743249,3565287,#636EFA,0.988911,351817,0.0982,10.4,04/12/20-CT,84.568088,20200412,CT,12035.0,29185.0,NaN,1654.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/12/2020 16:00,554.0,NaN,41220,41220,60.0,0.0,864.0,525.0,1389.0,04/12/20,1888.196759,0.003376,0.008186,29.20,0.011561,6.552094,4.615108,4.080446,4.465160,4.080446,3.218536,NaN,3.425010
7,Delaware,04/12/20,1625,35,191.0,166.878217,11103.0,190.0,2.153846,1140.214672,11.692308,973764,#636EFA,0.997342,204665,0.2156,8.1,04/12/20-DE,64.008594,20200412,DE,1479.0,9624.0,NaN,190.0,NaN,NaN,NaN,NaN,NaN,191.0,NaN,4/11/2020 13:30,33.0,NaN,11103,11103,0.0,0.0,0.0,0.0,0.0,04/12/20,986.794811,0.001519,0.009883,13.32,0.011402,5.988454,4.04

In [10]:
fig = px.line(main_df, x="date", y="Confirmed", color="state",
              line_group="state", hover_name="state")

fig.update_xaxes(title_font=dict(size=22, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=22, family='Arial', color='black'))
fig.show()

In [12]:
fig = px.line(main_df, x="date", y="People_Hospitalized", color="state",
              line_group="state", hover_name="state")

fig.update_xaxes(title_font=dict(size=22, family='Arial', color='black'))
fig.update_yaxes(title_font=dict(size=22, family='Arial', color='black'))
fig.show()